# How to Generate Earthdata Prerequisite Files

### Overview

This guide demonstrates how to generate three Earthdata prerequisite files (.netrc, .urs_cookies, .dodsrc) needed to access the GES DISC archives manually, or through Python. It uses native Python libraries to generate the files in either Windows/macOS/Linux environments and to place them in their appropriate folders. Additionally, it demonstrates how to retrieve an Earthdata Login token for use with the `Pydap` Python library.

### Example

<img src="https://raw.githubusercontent.com/nasa/gesdisc-tutorials/main/images/Prerequisite-Files-Diagram.png" alt="Prerequisite Files Diagram">

Figure 1: File and folder hierarchy showing where each Earthdata authentication file should be stored in your local filesystem.

### Prerequisites

This notebook was written using Python 3.8, and requires the following:
- Approval to access the GES DISC archives with your Earthdata credentials: https://disc.gsfc.nasa.gov/earthdata-login

### 1. Create Prerequisite Files Manually

- #### Create <code>.netrc</code> file:
    1. Make sure you have set up your [Earthdata account](https://disc.gsfc.nasa.gov/#top).
    2. Create a .netrc file in your home directory by following these steps:

    On **Mac/Linux**:
    1. <code>cd ~</code> or <code>cd $HOME</code>
    2. <code>touch .netrc</code>
    3. <code>echo "machine urs.earthdata.nasa.gov login < uid > password < password >" >> .netrc</code>, where < uid > is your username and < password > is your Earthdata Login password without the brackets.
    4. <code>chmod 0600 .netrc</code>, so only you can access it.

    On **Windows**:
    1. Open Notepad
    2. Enter (without quotes):<br><br>
       <code>machine urs.earthdata.nasa.gov login < uid > password < password ></code><br>
       Replace  <uid> with your username and <password> with your Earthdata Login password and remove the brackets.<br><br>
    3. Save as C:\\.netrc<br>

         

- #### Create <code>.urs_cookies</code> file:
    On **Mac/Linux**:
    1. <code>cd ~</code> or <code>cd $HOME</code>
    2. <code>touch .urs_cookies</code>

    On **Windows**:
    1. Open a run-command window by pressing <code>WinKey + R</code>
    2. Next, enter "cmd" in the text field and click "OK".
    3. Navigate to the home directory. In this guide, we place it under the C drive. You can navigate to the C drive by entering <code>C:</code>
    4. Finally, enter <code>NUL > .urs_cookies</code>.

    Note: you may get an 'Access denied' error. Enter 'dir' in Windows or ‘ls’ in Mac/Linux to verify that '.urs_cookies' file is listed in your directory.

- #### Create <code>.dodsrc</code> file:
    On **Mac/Linux**:
    1. <code>cd ~</code> or <code>cd $HOME</code>
    2. <code>touch .dodsrc</code>
    3. Open the .dodsrc file in a text editor and add the following lines, with the paths to your .netrc and .urs_cookies files: <br><br>
    HTTP.NETRC=< YourHomeDirectory >/.netrc<br>
    HTTP.COOKIEJAR=< YourHomeDirectory >/.urs_cookies<br><br>
    Replace < YourHomeDirectory > with the ABSOLUTE path of your home directory and remove the bracket.<br><br>

    On **Windows**:
    1. Open a run-command window by pressing <code>WinKey + R</code>
    2. Next, enter "cmd" in the text field and click "OK".
    3. Navigate to your root user directory.
    4. Enter <code>NUL > .dodsrc</code>
    5. Open the .dodsrc file in a text editor and add the following lines, with the paths to your .netrc and .urs_cookies files: <br><br>
    HTTP.NETRC=< YourHomeDirectory >/.netrc<br>
    HTTP.COOKIEJAR=< YourHomeDirectory >/.urs_cookies<br><br>
    Replace < YourHomeDirectory > with the ABSOLUTE path of your home directory and remove the bracket.<br><br>
    6. Copy the .dodsrc file to your *working* project directory.

### 2. Create prerequisite files using Python

The following Python code will prompt you for your Earthdata username and password before generating and storing these three files in their appropriate locations for Windows, Linux, or macOS operating systems. It can be run multiple times, and each time will overwrite existing credential files (if they already exist). This code can be run inside of this notebook or can be put into its own Python script.

**Note:** For Windows users, the <code>.dodsrc</code> file is also required in your working directory, which may require re-running this code in other project directories, or manually copying the <code>.dodsrc</code> file into any other project directories that access GES DISC data. The code below will automatically check if it is run inside of a Windows environment and will copy <code>.dodsrc</code> into your current working directory.

In [ ]:
from subprocess import Popen
from getpass import getpass
import platform
import os
import shutil

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

homeDir = os.path.expanduser("~") + os.sep

with open(homeDir + '.netrc', 'w') as file:
    file.write('machine {} login {} password {}'.format(urs, getpass(prompt=prompts[0]), getpass(prompt=prompts[1])))
    file.close()
with open(homeDir + '.urs_cookies', 'w') as file:
    file.write('')
    file.close()
with open(homeDir + '.dodsrc', 'w') as file:
    file.write('HTTP.COOKIEJAR={}.urs_cookies\n'.format(homeDir))
    file.write('HTTP.NETRC={}.netrc'.format(homeDir))
    file.close()

print('Saved .netrc, .urs_cookies, and .dodsrc to:', homeDir)

# Set appropriate permissions for Linux/macOS
if platform.system() != "Windows":
    Popen('chmod og-rw ~/.netrc', shell=True)
else:
    # Copy dodsrc to working directory in Windows  
    shutil.copy2(homeDir + '.dodsrc', os.getcwd())
    print('Copied .dodsrc to:', os.getcwd())


### 3. ***Pydap Only:*** Create Earthdata Login Token using Python

For some Python libraries, specifically `pydap`, an Earthdata Login token can be generated and saved to a variable in place of Earthdata prerequisite files. The following code snippet will prompt you for your Earthdata username and password, and will save the token to a variable. Uncomment the `print` line if you wish to display the token inside this notebook.

In [ ]:
import requests
from base64 import b64encode
from getpass import getpass

# Earthdata Login URL for obtaining the token, and creating one if it doesn't exist
url = 'https://urs.earthdata.nasa.gov/api/users/find_or_create_token'

# Earthdata Login credential prompts
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Encode credentials using Base64, and prompt for credentials
credentials = b64encode(f"{getpass(prompt=prompts[0])}:{getpass(prompt=prompts[1])}".encode('utf-8')).decode('utf-8')

# Headers with the Basic Authorization
headers = {
    'Authorization': f'Basic {credentials}'
}

# Make the POST request to get the token
response = requests.post(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON to get the token and expiration date
    token_info = response.json()
    print("Token retrieved successfully")
    # Uncomment the following line to print the token inside the notebook:
    # print(token_info)

else:
    print("Failed to retrieve token:", response.text)